# Absorption spectra in ORCA (TD-DFT)

In [3]:
import os,shutil,sys
import ase
from ase.io import read,write,iread
from ase.visualize import view
from ase.optimize import FIRE, BFGS
from ase.neb import NEBTools
from ase.calculators.aims import Aims
from ase.calculators.orca import ORCA # comment this if orca.ase hasn't been set up yet

## Performing the TDDFT calculation - .inp setup, to be run through SLURM/command line.
The below cell writes a .py file which can be used to generate the input file, can be used on HPC this way with a SLURM submit script

In [10]:
%%writefile tddft_input_gen.py

import sys
def tddft_input_gen(functional, basis, roots, xyz, procs):
    inp = open('./tddft_calc.inp', 'w+')
    inp.write('! RKS '+str(functional)+' '+str(basis)+' 'str(basis)+'/c def2/J RIJCOSX \n'
              '%pal nprocs '+str(procs)+' end \n'
              '%tddft nroots '+str(roots)+' maxdim 10 end \n'
              '* xyzfile 0 1 '+str(xyz)+' \n')
    inp.close()
tddft_input_gen(sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4], sys.argv[5])
print('Generated file: tddft_calc.inp')

Overwriting tddft_input_gen.py


## Performing the TDDFT calculation - ASE implementation

In [11]:
%%writefile tddft_orca_ase.py
def tddft_calc(functional, basis, roots, system, procs):
    system = read(str(system))
    system.calc = ORCA(label='orca',
    maxiter = 200,
    charge = 0, mult=1,
    orcasimpleinput = str(functional)+' '+str(basis),
    orcablocks = '%pal nprocs '+str(procs)+' end \n'
    '%tddft nroots '+str(roots)+' maxdim 10 end \n')
    try:
        print(system.get_potential_energy())
    except:
        print('Calculation was not successful')


tddft_calc(sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4], sys.argv[5)

Writing tddft_orca_ase.py


## Spectra and visualization

In [52]:
%%writefile generate_abs_spectra.py
import sys
def generate_abs_spectra(outfilename, startwavelength, endwavelength):
    import subprocess,time,sys
    import matplotlib
    import matplotlib.pyplot as plt
    bashcmd = 'which orca'
    process = subprocess.Popen(bashcmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    os.system(output[0:-1]+'_mapspc '+outfilename+' ABS -x0'+str(1e+6/startwavelength)+' -x1'+str(1e+6/endwavelength))
    time.sleep(2)
    print('Spectra files generated')

generate_abs_spectra(sys.argv[1], sys.argv[2], sys.argv[3])

Overwriting generate_abs_spectra.py


/home/chem/msuccm/software/orca412/orca_mapspc


0